In [4]:
import numpy as np
import pandas as pd
import copy as cp

In [5]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [6]:
class Neuron:
  def __init__(self, x=None, w=None, function=sigmoid):
    self.x = x
    self.u = None
    self.pd = None
    self.function = function
    
    if type(self.x) is float or type(self.x) is int:
      # Bias ou entrada
      self.w = None
      self.g = self.x
    elif type(self.x) is list:
      # Normal
      self.w = w if type(w) is np.ndarray else np.random.random(len(self.x))
      self.u = self.x @ self.w
      self.g = self.function(self.u)
    else:
      # Não-inicializado
      self.w = w if w is None or type(w) is np.ndarray else np.random.random(w)
      self.g = None
      
  def update(self, new_x):
      self.x = new_x
      if not self.is_bias():
        self.u = self.x @ self.w
        self.g = self.function(self.u)
      else:
        self.g = self.x
  
  is_bias = lambda self: True if self.w is None else False
  
  __repr__ = lambda self: '{} @ {} -> {}'.format(self.x, self.w, self.g)

In [21]:
class MLP:
  def __init__(self, topology):
    self.topology = topology
    
    self.init_neurons()   
    
  def init_neurons(self):
    # Camada de entrada
    self.layers = [[Neuron() for i in range(self.topology[0])]]
    self.layers[0] += [Neuron(1)] # Bias 
    
    # Camadas ocultas e de saída
    for i in range(1, len(self.topology)):
      self.layers += [[Neuron(w=len(self.layers[i - 1])) for j in range(self.topology[i])]]
      if i != len(self.topology) - 1: self.layers[i] += [Neuron(1)] # Bias

  def update_neurons(self): 
    # Camada de entrada
    [self.layers[0][i].update(self.inputs[i]) for i in range(len(self.inputs))]
    
    # Camadas ocultas e de saída
    for i in range(1, len(self.layers)):
      [n.update([e.g for e in self.layers[i - 1]]) for n in self.layers[i] if not n.is_bias()]
        
  def update_weights(self):
    # Camada de saída
    for i in range(len(self.layers[-1])):
      # Derivada parcial
      neuron = self.layers[-1][i]
      neuron.pd = - (self.desired[i] - neuron.g) * neuron.g * (1 - neuron.g)
      # Pesos
      for j in range(len(neuron.w)):
        neuron.w[j] -= self.learning_rate * neuron.pd * self.layers[-2][j].g
          
    # Camadas ocultas
    for i in range(len(self.layers) - 2, 0, -1):
      for j in range(len(self.layers[i]) - 1):
        # Derivada parcial
        neuron = self.layers[i][j]
#         print(neuron)
        descendents = sum([n.pd * n.w[j] for n in self.layers[i + 1] if not n.is_bias()])
        neuron.pd = descendents * neuron.g * (1 - neuron.g)
        # Pesos
        for k in range(len(neuron.w)):
          neuron.w[k] -= self.learning_rate * neuron.pd * neuron.x[k]
  
  def update_errors(self):
    out_layer = self.layers[-1]
    out_length = len(out_layer)
    self.errors = [(out_layer[i].g - self.desired[i]) ** 2 for i in range(out_length)]
    self.total_error = sum(self.errors) / 2
    
  def train(self, test_data, learning_rate=.1, precision=10**-6):
    self.learning_rate = learning_rate
    self.precision = precision
    self.epochs = 0
    current_mse = 0
    last_mse = 0
    
    while abs(current_mse - last_mse) > self.precision or self.epochs == 0:
      last_mse = current_mse
      
      errors_sum = 0
      for index, sample in test_data.iterrows():
        self.inputs = sample[:self.topology[0]].tolist()
        self.desired = sample[self.topology[0]:].tolist()
        self.update_neurons()

        self.update_errors()
        errors_sum += self.total_error

        self.update_weights()

      self.update_neurons()
      self.epochs += 1
      
      current_mse = errors_sum / test_data.shape[0]
        
  def test_individual(self, sample):
    layers = cp.deepcopy(self.layers)
    
    [layers[0][i].update(sample[i]) for i in range(len(sample))]
    for i in range(1, len(layers)):
      [n.update([e.g for e in layers[i - 1]]) for n in layers[i] if not n.is_bias()]
      
    return [n.g for n in layers[-1]]
  
  def __repr__(self):
    result = 'Topologia: {}\t'.format(self.topology)
    result += 'Épocas: {}\n\n'.format(self.epochs)
    result += 'PESOS\n'
    for i in range(1, len(self.layers)):
      result += 'Camada {}\n'.format(i)
      for j in range(len(self.layers[i])):
        neuron = self.layers[i][j]
        if not neuron.is_bias():
          result += 'Neurônio {:2d}: {}\n'.format(j, neuron.w)
      result += '\n'

    return result

In [30]:
df = pd.read_csv('../../datasets/ressonancia.csv')
df = df.iloc[:2]

In [31]:
aan = MLP([3, 2, 1])
aan.train(df)
aan

Topologia: [3, 2, 1]	Épocas: 244

PESOS
Camada 1
Neurônio  0: [0.84458428 0.32885161 0.62661012 0.27175785]
Neurônio  1: [0.13855793 0.55085711 0.39489494 0.37044522]

Camada 2
Neurônio  0: [0.57306673 0.06936451 0.55377107]


In [124]:
sample = 199
aan.test_individual(df.values[sample][:-1]), df.values[sample][-1]

([0.5399474860150307], 0.5625)